<a href="https://colab.research.google.com/github/alvarezpablo/GPT/blob/main/Resumen_de_NC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook para Analizar y Resumin Nueva Constitución de la República de Chile.



In [ ]:
#Instalación de Librerías Python
!pip install openai
!pip install tiktoken
!pip install langchain
!pip install pdfplumber

In [ ]:
import openai
import os
import math
import pdfplumber
from langchain.text_splitter import CharacterTextSplitter
from openai import OpenAI

In [ ]:
# Configuración de OpenAI

openai.api_key = ""
os.environ["OPENAI_API_KEY"] = openai.api_key
client = OpenAI()


In [ ]:

# Configuración
CONFIG = {
  'engine_resumen': 'gpt-3.5-turbo',
  'engine_final': 'gpt-4-1106-preview',
  'max_tokens': 4096,
  'temperature': 0.7,
  'longitud': 2000,
  'palabras_por_pagina' : 300
}





Se probaron dos enfoques, uno de segmentar en texto por chucks y resumir cada uno, para posteriormente hacer un resumen Global. El segundo fue separar por capítulos y resumir cada Uno.

El segundo enfoque entrega un texto de mejor calidad.

In [ ]:
#Funciones para realizar distintas tareas

def extraer_texto_de_pdf(ruta_archivo):
  texto = ""
  with pdfplumber.open(ruta_archivo) as pdf:
    for pagina in pdf.pages:
      texto += pagina.extract_text()
  return texto


def dividir_en_segmentos(texto, config):

  longitud = config['longitud']

  segmentos = []
  while texto:
    parte = texto[:longitud]
    if len(texto) > longitud and " " in parte:
      ultimo_espacio = parte.rfind(" ")
      parte = texto[:ultimo_espacio]
      texto = texto[ultimo_espacio:].lstrip()
    else:
      texto = ""
    segmentos.append(parte)
  return segmentos


def resumir_segmento(segmento, config):
  print(f"segmento {segmento}......")
  prompt = "Por favor, resuma el texto siguiente legal de forma clara y precisa."
  prompt = "Por favor, resuma el texto siguiente legal de forma clara y precisa. Usted es un abogado consitucionalista y debe hacer un resumen para el publico general no experto. El texto es la propuesta de una nueva consitucion para la republica de Chile y lo debe hacer en un maximo de 300 palabras"
  engine = config['engine_resumen']

  try:
    chat_completion = client.chat.completions.create(
      model=engine,
      messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": segmento}
      ]
    )

    respuesta = chat_completion.choices[0].message.content
    print(f"resumen {respuesta}......")
    return respuesta.strip()

  except Exception as e:
    print(f"Error al llamar OpenAI: {e}")
    return ""


def concatenar_resumenes(resumenes):
  return " ".join(resumenes)


def reducir_a_resumen_final(resumen_completo, prompt, config):
  prompt = """

Por favor, genere un resumen comprensivo de máximo 5 páginas sobre los contenidos más relevantes del texto, despues de 'Este es el texto de la propuesta de nueva Constitución para la República de Chile' . El resumen debe estar escrito en un lenguaje claro y accesible para el público general. Debe cubrir al menos los siguientes temas:

Principios fundamentales y derechos individuales
Derechos sociales
Sistema político y de gobierno
Poder Legislativo
Poder Ejecutivo
Poder Judicial
Órganos autónomos como el Banco Central y la Contraloría
Organización territorial del Estado
Reforma y cambio constitucional
Para cada tema, por favor resuma los aspectos más relevantes de la propuesta constitucional, citando artículos o capítulos cuando sea pertinente. Cuando mencione conceptos legales o técnicos, explíquelos de manera sencilla. No incluya detalles menores o irrelevantes. El resumen debe ser objetivo y equilibrado, sin sesgos a favor o en contra de la propuesta. Por favor, respete el límite de 5 páginas

  """

  resumen_completo = "Este es el texto de la propuesta de nueva Constitución para la República de Chile: " + resumen_completo

  engine = config['engine_final']

  try:
    response = client.chat.completions.create(
      model = engine,
      messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": resumen_completo}
      ]
    )
    return response.choices[0].message.content.strip()

  except Exception as e:
    print(f"Error al llamar OpenAI: {e}")
    return ""

def reducir_a_resumen_final_v2(resumen_completo, prompt, config):
  prompt = """

Por favor, genere un resumen comprensivo de máximo 5 páginas sobre los contenidos más relevantes del texto, despues de 'Este es el texto de la propuesta de nueva Constitución para la República de Chile' . El resumen debe estar escrito en un lenguaje claro y accesible para el público general. Debe cubrir al menos los siguientes temas:

Principios fundamentales y derechos individuales
Derechos sociales
Sistema político y de gobierno
Poder Legislativo
Poder Ejecutivo
Poder Judicial
Órganos autónomos como el Banco Central y la Contraloría
Organización territorial del Estado
Reforma y cambio constitucional
Para cada tema, por favor resuma los aspectos más relevantes de la propuesta constitucional, citando artículos o capítulos cuando sea pertinente. Cuando mencione conceptos legales o técnicos, explíquelos de manera sencilla. No incluya detalles menores o irrelevantes. El resumen debe ser objetivo y equilibrado, sin sesgos a favor o en contra de la propuesta. Por favor, respete el límite de 5 páginas

  """

  resumen_completo = "Este es el texto de la propuesta de nueva Constitución para la República de Chile: " + resumen_completo

  engine = config['engine_final']

  try:
    response = client.chat.completions.create(
      model = engine,
      messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": resumen_completo}
      ]
    )
    return response.choices[0].message.content.strip()

  except Exception as e:
    print(f"Error al llamar OpenAI: {e}")
    return ""


def calcular_num_paginas(texto, config):
  palabras_por_pagina = config ['palabras_por_pagina']
  num_palabras = len(texto.split())
  return math.ceil(num_palabras / palabras_por_pagina)


def generar_prompt(num_paginas, config):
  PALABRAS_POR_PAGINA = config['palabras_por_pagina']
  return f"Por favor, resuma el texto siguiente en un máximo de {num_paginas} páginas con {PALABRAS_POR_PAGINA} palabras por página:"


def resumir_segmentos(texto, config):

  print("Dividiendo texto en segmentos...")
  segmentos = dividir_en_segmentos(texto, config)

  print("Resumiendo segmentos...")
  resumenes = [resumir_segmento(seg, config) for seg in segmentos]

  print("Concatenando resúmenes...")
  return concatenar_resumenes(resumenes)


def resumen_capitulos(texto):
    if not texto:
        raise ValueError("El texto no puede estar vacío")

    engine = CONFIG['engine_final']
    max_tokens = CONFIG['max_tokens']

    prompt_inicial = "procese el texto siguiente y espere instrucciones"
    prompt_final = """

    Por favor, genere un resumen comprensivo sobre los contenidos del capitulo indicado más relevantes del texto El resumen debe estar escrito en un lenguaje claro y accesible para el público general.

    Para cada tema, por favor resuma los aspectos más relevantes de la propuesta constitucional, citando artículos o capítulos cuando sea pertinente. Cuando mencione conceptos legales o técnicos, explíquelos de manera sencilla. No incluya detalles menores o irrelevantes. El resumen debe ser objetivo y equilibrado, sin sesgos a favor o en contra de la propuesta.


    """

    resumen_completo = "'Este es el texto de un capítulo de propuesta de nueva Constitución para la República de Chile': " + texto

    try:
        response = client.chat.completions.create(
            model = engine,
            max_tokens=max_tokens,
            messages=[
                {"role": "system", "content": "Tu eres un abogado constitucionalista. Vas a analizar el texto que sigue."},
                {"role": "user", "content": prompt_inicial},
                {"role": "assistant", "content": resumen_completo},
                {"role": "user", "content": prompt_final}
            ]
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Se produjo un error al generar el resumen: {e}")
        return ""





def generar_resumen_final(resumen_completo, num_paginas, config):

  print("Calculando número de páginas...")
  paginas_actuales = calcular_num_paginas(resumen_completo, config)

  print("Generando prompt...")
  prompt = generar_prompt(num_paginas, config)

  print("Obteniendo resumen final...")
  return reducir_a_resumen_final(resumen_completo, prompt, config)


def resumir_a_num_paginas(texto, num_paginas, config):

  resumen_completo = resumir_segmentos(texto, config)

  return generar_resumen_final(resumen_completo, num_paginas, config)




In [ ]:
# Se lee texto a partir de PDF

texto = extraer_texto_de_pdf('PROPUESTA-DE-NUEVA-CONSTITUCION-POLITICA-DE-LA-REPUB-LICA.pdf')



In [ ]:
text_respaldo = texto

In [ ]:
import re

def roman_to_int(num_romano):
  conversiones = {
    'I': 1,
    'V': 5,
    'X': 10,
    'L': 50,
    'C': 100,
    'D': 500,
    'M': 1000
  }

  num = 0
  for i in range(len(num_romano)):
    valor = conversiones[num_romano[i]]
    if i + 1 < len(num_romano) and conversiones[num_romano[i + 1]] > valor:
      num -= valor
    else:
      num += valor
  return num

# Texto de entrada
texto = text_respaldo

# Buscar posición final para borrar
idx_borrar = texto.find("DISPOSICIONES TRANSITORIAS\n")

if idx_borrar > -1:
  texto = texto[:idx_borrar]

# Encontrar inicio del primer capítulo
inicio_texto = texto.find("CAPÍTULO I\n")

if inicio_texto > -1:
  texto = texto[inicio_texto:]



# Expresión para números romanos
pattern_romano = r"[IVXLCDM]+"

# Buscar títulos
pattern_titulo = r"(CAPÍTULO\s+(?:"+pattern_romano+"))\n"

capitulos = []

for match in re.finditer(pattern_titulo, texto):

  num_romano = match.group(1).split(" ")[1]
  num_arabigo = roman_to_int(num_romano)

  inicio = match.start()

  capitulos.append((num_arabigo, inicio))

# Ordenar por número
capitulos.sort(key=lambda x: x[0])



In [ ]:
print(capitulos)

In [ ]:
# Extraer capítulos

texto_capitulos = []

inicios = [c[1] for c in capitulos]

for i in range(len(inicios)-1):

  inicio = inicios[i]
  fin = inicios[i+1]

  capitulo = texto[inicio:fin]

  num_cap = i+1
  texto_capitulos.append(capitulo)
  with open("capitulo_"+str(num_cap)+".txt", "w") as f:
    f.write(capitulo)

# Obtener último capítulo
inicio = inicios[-1]
fin = len(texto)

ultimo_cap = texto[inicio:fin]

num_cap = len(inicios)

texto_capitulos.append(ultimo_cap)
with open("capitulo_"+str(num_cap)+".txt", "w") as f:
  f.write(ultimo_cap)

In [ ]:
for i in range(len(texto_capitulos)):
  _texto = resumen_capitulos(resumen_capitulos("CAPÍTULO {i+1}\n" + texto_capitulos[i]))
  print(f"Resumen Capítulo {i+1}")
  print(f"{_texto}")